In [24]:
import requests
import time
import re
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import http.cookiejar
import os 
import sys
import ijson
import io
from bson.code import Code
import json
import string
from datetime import datetime

### 1.Scrape Nasdaq to get all the news link, title and dynamic api link

In [35]:
#The website url does not get the data by get request, it calls an api which can be found by inspecting
headers = {
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-Encoding":"gzip, deflate",
"Accept-Language":"en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
"Connection":"keep-alive",
"Host":"www.nasdaq.com",
"Referer":"http://www.nasdaq.com",
"Upgrade-Insecure-Requests":"1",
'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'} 

result_links=[]
result_titles=[]
for pg in range(1,50): #page number
    api_url = 'https://www.nasdaq.com/api/v1/search?q=tesla&offset='+str((pg-1)*10)  # get dynamic api link
    response=requests.get(api_url,headers=headers)
    json_result = json.loads(response.content) #the result is a json file 
    html_result=json_result['items'] #parse the json file 
    no_of_result=0 #count number of result 
    for el in html_result:
        time.sleep(3) 
        soup=BeautifulSoup(el,'html.parser')  ##parse the html beautifully
        no_of_result=no_of_result+1
        print(no_of_result," results")
        
        #find each search result title,link, and timestamp
        result_title_tag=soup.select("h2.search-result__title>a")[0] 
        result_title=result_title_tag.get("title")
        partial_link=result_title_tag.get("href") #link is only partial
        try:
            date_stamp_string=re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2}',partial_link)[0]
            date_stamp=datetime.strptime(date_stamp_string, '%Y-%m-%d') #transfer to date object
            date_stamp_bchmark=datetime.strptime('2019-10-01','%Y-%m-%d') #2019.10.01 is the time stock price start to soar
        except:
            continue
        
        #skip the symbol results which are not news
        result_eyebrow=soup.select("div.search-result__eyebrow")[0]
        #skip the ones that are not news or the ones that are earlier than 2019.10.01
        if result_eyebrow.text!="Symbols" and date_stamp>=date_stamp_bchmark: 
            full_link="https://www.nasdaq.com"+str(partial_link)
            result_links.append(full_link)
            result_titles.append(result_title)
            print(result_title)
            print(full_link)
    if response.status_code!=200:
        raise ValueError("Invalid Response Received From Webserver")
    #print(result)

1  results
2  results
3  results
Tesla, Tesla, Tesla: Where Does The Rally End?
https://www.nasdaq.com/articles/tesla-tesla-tesla%3A-where-does-the-rally-end-2020-02-06
4  results
5  results
6  results
7  results
Tesla Collar Update
https://www.nasdaq.com/articles/tesla-collar-update-2019-12-26
8  results
9  results
10  results
1  results
Tesla Unveils Cybertruck 
https://www.nasdaq.com/articles/tesla-unveils-cybertruck-2019-11-22
2  results
3  results
4  results
5  results
6  results
7  results
8  results
9  results
10  results
1  results
2  results
Tesla in 4 Charts
https://www.nasdaq.com/articles/tesla-in-4-charts-2019-10-06
3  results
Is Tesla Worth It?
https://www.nasdaq.com/articles/is-tesla-worth-it-2019-11-06
4  results
5  results
Comparing Tesla With Toyota
https://www.nasdaq.com/articles/comparing-tesla-with-toyota-2020-02-11
6  results
7  results
8  results
9  results
10  results
1  results
2  results
3  results
4  results
5  results
6  results
7  results
Tesla Stock Worth $

## 2. download the links 

In [36]:
#need to get the cookie 
session_requests = requests.session()
# going to the home page while logged in  
r2=session_requests.get('https://www.nasdaq.com/',headers=headers)
cookie=r2.cookies.get_dict()
print("cookie is :",cookie)

for link in result_links:
    ##download each html  GET requests
    response=requests.get(link,cookies=cookie,headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser') 
#   print(soup)
#   write file
    no_of_news=result_links.index(link)
    title=str(result_titles[no_of_news]).translate(str.maketrans(' ','_',string.punctuation))
    with open("tesla_news_"+str(no_of_news+1)+"_"+str(title)+".htm","w",encoding='utf-8') as file:
        html_unicode=UnicodeDammit(str(soup)).unicode_markup
        file.write(html_unicode)

cookie is : {'AKA_A2': 'A', 'ak_bmsc': 'BB02AB9E3BF4994D3A877540F7AC1929C7EFB71D8164000019D55D5EB6D1C160~plj14IxwDPFoC2+Ok3iSP8uCFFZVNytjMWYGNhjLCyTlS0j/U5XIv1de4W9Gt9aOSlsC5KXrBQ3EgeP8QIA59UCAz5be9kLDSxCyS60LKgjXW8Cl6MYLV1HDjvu3Arkd6OKLrXLKDClUaHTwrBg7NkAeRK6i0/k6/Mmni3psgTYWwwQpntqU4aBRYgsMVTDaoIoJD2vmtk/jaSa+Tjqg7iprbH+YvXysL6Sb47PbhzAzw='}


## 3. Parse and get the content of the news 

In [37]:
#    with open(filename, 'r',encoding="utf-8") as file:
#        text = file.read()
#    soup = BeautifulSoup(response.text, 'html.parser') 
#    print(no_of_result)
#    print(soup)
directory = os.getcwd() #get the directory
for filename in os.listdir(directory):
    if filename.endswith('.htm') == False:
        continue
    try:
        with open(os.path.join(directory, filename), 'r',encoding="utf-8") as file:
            text = file.read()
            soup = BeautifulSoup(text, 'html.parser') 
            #print(soup)
            #get paragraph
            paragraphs = soup.findAll('p') 
        if not paragraphs:
            paragraph=None
        else:
            print(filename)
            for el in paragraphs[1:]: #first <p> label is meaningless
                if el.text=="The views and opinions expressed herein are the views and opinions of the author and do not necessarily reflect those of Nasdaq, Inc.":
                    break

                print(el.text) 
    except:
        print("file "+filename,sys.exc_info()[0])

   #seller names

tesla_news_10_Nike_Tesla_Lead_Stocks_Higher.htm
U.S. stocks wrangled in a notable win for the first trading day in February, quickly writing off Friday's drop. A solid 3.6% lift out of Nike (NKE) was the biggest driver for the Dow today, guiding the index 143 points higher by session end. Tesla (TSLA) was also big winner, soaring 20%. Although earnings activity was light today, the highly anticipated Alphabet (GOOGL) report is due out this evening.
 
 
 
Continue reading for more on today's market, including:
The Dow Jones Industrial Average (DJI - 28,399.81) added 143.8 points, or 0.5% for the day. Nike paced the 21 winners  with a 3.1% gain, while Verizon (VZ) lead the nine losers with a 2.3% drop. 
The S&P 500 Index (SPX - 3,248.92) tacked on 23.4 points, or 0.7%. Meanwhile, the Nasdaq Composite (IXIC - 9,273.40) finished 122.5 points, or 1.3%, higher today.
The Cboe Volatility Index (VIX - 17.97) shed 0.9 point, or 4.6%.
 
 
 




 




Data courtesy of Trade-Alert
Coronavirus head

tesla_news_14_Why_Tesla_Stock_Jumped_Tuesday.htm
Shares of Tesla (NASDAQ: TSLA) jumped on Tuesday, rising 6.5% as of 11:10 a.m. EST.
The gain follows one analyst's decision to significantly increase his 12-month price target for the stock based on the company's recent execution and his expectations for more sharp sales growth from the electric-car company in the coming years.
Image source: The Motley Fool.
New Street analyst Pierre Ferragu gave Tesla stock an $800 12-month price target on Tuesday. This is up from his previous 12-month price target of $530. Supporting the stock's 80% gain over the last year has been the company's "technological dominance, strong sustained demand, Tesla's ability to execute," said Ferragu in a note to investors. 
Looking ahead, Ferragu thinks Tesla's annual deliveries can increase from a run rate of about 450,000 today to 2 million to 3 million units by 2025. In addition, the analyst believes the company will boast "industry leading margins."
Tesla stock

Shares of electric-car company Tesla (NASDAQ: TSLA) took a hit on Thursday, falling as much as 7.8%. As of 11:36 a.m. EDT, the stock was down 6.6%.
The stock's decline follows Tesla's third-quarter update on vehicle production and deliveries. Shares are likely down because deliveries were below analysts' average forecast for the period.
Model 3. Image source: Tesla.
Tesla said it delivered a record 97,000 vehicles in its third quarter. This was up from the company's previous quarterly record, which was set in the second quarter of 2019, of 95,356 vehicles. But it was below what analysts were looking for. On average, analysts were expecting Tesla to deliver 99,000 vehicles during the period. 
Total vehicle deliveries were up 16% year over year. Model 3 deliveries continued to drive Tesla's growth, accounting for 82% of the quarter's deliveries.
Management said it produced a record 96,155 vehicles during the period, up from 87,048 vehicles in Q2.
Demand for the company's vehicles remains

tesla_news_24_Why_Tesla_Stock_Fell_Thursday.htm
Shares of electric-car maker Tesla (NASDAQ: TSLA) fell on Thursday, declining as much as 5.1%. As of 11:14 a.m. EST, however, the stock was down 3.8%.
The stock's decline came as Morgan Stanley analyst Adam Jonas issued an underweight rating for the stock. This, which is the equivalent of a sell rating, follows a huge run-up in the stock price. Jonas believes shares are now overvalued.
Image source: Getty Images.
In a note to investors on Thursday, Jonas increased his 12-month price target on the stock from $250 to $360 but downgraded his rating on the stock from hold to sell.
The analyst noted that though he has been "encouraged by Tesla's execution" recently, he believes "investors will be presented with more attractive opportunities to own the stock in the future."
Shares have skyrocketed in recent months. Even after the stock's decline on Thursday, shares are up 93% in the past three months.
It's true that Tesla's valuation has become

tesla_news_30_Is_There_More_Spark_in_Tesla.htm
Tesla TSLA shares have surged 66% over the past month taking its valuation above that of General Motors GM, Volkswagen and Fiat Chrysler FCAU combined. Shorts are also at a one-year low of 15% (according to S3 data) as easy liquidity increased momentum in a stock that has seen a steady flow of positive news, including several upgrades and price target increases from sell-side analysts.

And despite the soaring prices, the eccentric CEO Elon Musk (and foremost Tesla bull out there) increased his stake to 34.1 million shares or an 18.5% share of the company. And not one to pass up an opportunity, especially given the investments the company has scheduled to bring new models and factories online, he initiated another public share issue. Despite the resultant dilution, 2.65 million shares were lapped up and the deal's underwriters exercised in full the option to buy another 397,500 shares, netting $2.31 billion for the company.So the big quest

tesla_news_33_Why_is_Tesla_Stock_Jumping_Today.htm
Shares of electric car maker Tesla TSLA closed up almost 4% today, briefly touching a new all-time high of $395.00. And, TSLA is up almost 60% over the past three months. TSLA is now on pace for its best quarter since Q3 2013.
What’s behind this bullishness? Well, Tesla’s surprise profit in last quarter has really boosted the stock investor optimism. And then you have the progress at the company’s Shanghai Gigafactory, where production is slowly ramping up and is overally a positive catalyst for Tesla. A phase one trade deal between the U.S. and China, announced last week, would also benefit Tesla’s operations in the country; CEO Elon Musk believes that the Chinese market will be key for Tesla’s future.
Musk also recently revealed plans for a fourth gigafactory in Berlin, which will help Tesla avoid any export complications moving its cars to Europe. Plus, the company just unveiled two new car models: the crossover SUV Model Y and the 

tesla_news_37_Will_Tesla_Continue_to_Surge_Higher.htm
As of late, it has definitely been a great time to be an investor in Tesla, Inc. TSLA. The stock has moved higher by 18% in the past month, while it is also above its 20 Day SMA too. This combination of strong price performance and favorable technical, could suggest that the stock may be on the right path.
We certainly think that this might be the case, particularly if you consider TSLA’s recent earnings estimate revision activity. From this look, the company’s future is quite favorable; as TSLA has earned itself a Zacks Rank #2 (Buy), meaning that its recent run may continue for a bit longer, and that this isn’t the top for the in-focus company. You can see the complete list of today’s Zacks #1 Rank stocks here.
More Stock News: This Is Bigger than the iPhone!
It could become the mother of all technological revolutions. Apple sold a mere 1 billion iPhones in 10 years but a new breakthrough is expected to generate more than 27 billi

tesla_news_40_How_Does_Tesla_Spend_Its_Money.htm
Electric vehicle major Tesla (NASDAQ: TSLA) had a strong 2019, with its stock price soaring by close to 35% through the year driven by growing revenues and better cost management, which has enabled to company to post surprise quarterly profits. In this analysis, we take a look at the key drivers of Tesla’s Expenses and Net Margins.
View our interactive dashboard analysis on Tesla Expenses: How Does Tesla Spend Its Money?
TAKEAWAYS
A Closer Look At Tesla’s Key Expenses 
Tesla’s Total Expenses Have Increased from $6 billion in 2015 to $22.5 billion in 2018, driven by the launch of the Model 3. We expect Total Expenses to grow to ~$28 billion by 2020
 
For more details and charts on Tesla’s expenses, view our comprehensive dashboard analysis
What’s behind Trefis? See How it’s Powering New Collaboration and What-Ifs
For CFOs and Finance Teams | Product, R&D, and Marketing Teams
More Trefis Data
Like our charts? Explore example interactive da

tesla_news_6_Comparing_Tesla_With_Toyota.htm
Tesla (NASDAQ:TSLA) stock is up by almost 2.5x over the last three months, making it the world’s second most valuable automaker, behind Toyota, which in many respects is the benchmark of profitability and efficiency in the auto industry. In this analysis, we compare Tesla’s key revenue, margin, and operating metrics with Toyota’s.
Note: Toyota’s FY ends March, while Tesla’s ends in December. We have excluded numbers from Toyota’s financing business for the purpose of this analysis.
 View our complete dashboard analysis on How Does Tesla Compare With Toyota?
 
#1. Key Revenue Metrics
#1.1 Automotive Revenues
#1.2 Automotive Volumes and Average Pricing
#2. Gross Margins & Operating Expenses
#2.1 Gross Margins
#2.2 Operating Expenses
#3. Other Operating Metrics
#3.1 Working Capital Ratios
#3.2 Vehicles Delivered Per Full-Time Employee
#3.3 Fixed Asset Turnover & Capital Expenditure
See all Trefis Price Estimates and Download Trefis Data here
Wh